# Hamming weight and Pearson Correlation

This notebook demonstrates how to guess a floating number using Hamming Weight model.

We generate a random secret floating number. Then we use hamming weight model to guess the number.

In [1]:
import pathlib
TEST_RESULT_DIR = './results'

pathlib.Path(TEST_RESULT_DIR).mkdir(parents=True, exist_ok=True) 

In [2]:
import sys
sys.path.append("../")

from pybatina.weight_recovery import *
from pybatina.utils import *

In [9]:
import numpy as np

mantissa_nbits = 12
guess_range=(-5.0, 5.0)
# the number of guess values in the guess range
secret_number = np.random.uniform(guess_range[0], guess_range[1]) 
secret_number = -0.498121310
secret_number = int_to_float(float_to_int(secret_number))

print('secret_number =', secret_number)

secret_number = -0.49812132120132446


## Receover the weight number in step-by-step

### Correlation of mantissa

In [ ]:
guess_numbers = build_guess_values(component='mantissa', mantissa_nbits=mantissa_nbits, guess_range=guess_range)
known_inputs = build_input_values(mantissa_nbits=mantissa_nbits, component='mantissa')
secret_hw = np.vectorize(hamming_weight)(known_inputs * secret_number)
pd.Series(known_inputs).sort_values().reset_index(drop=True).plot(figsize=(15,6), marker='.')

In [ ]:
%%time
mantisa_corr = compute_corr_numbers(secret_hw, known_inputs, guess_numbers)

In [ ]:
ax = mantisa_corr.sort_index().plot(figsize=(15,6), label='correlation')
ax.vlines(mantisa_corr.idxmax(), mantisa_corr.min(), mantisa_corr.max(), color='green', label='max correlation')
ax.vlines(secret_number, mantisa_corr.min(), mantisa_corr.max(), color='red', label='secret value')
ax.legend()
ax.grid()

In [ ]:
mantisa_corr.sort_values(ascending=False).iloc[:10]

### Correlation exponent

In [ ]:
known_inputs = build_input_values(component='exponent')
secret_hw = np.vectorize(hamming_weight)(known_inputs * secret_number)
guess_numbers = build_guess_values(component='exponent', numbers=mantisa_corr.sort_values(ascending=False).index[:10], guess_range=guess_range)
pd.Series(guess_numbers).sort_values().reset_index(drop=True).plot(figsize=(12,6), marker='.')

In [ ]:
%%time
mantisa_exp_corr = compute_corr_numbers(secret_hw, known_inputs, guess_numbers)

In [ ]:
ax = mantisa_exp_corr.sort_index().plot(figsize=(15,6), label='correlation')
ax.vlines(mantisa_exp_corr.idxmax(), mantisa_exp_corr.min(), mantisa_exp_corr.max(), color='green', label='max correlation')
ax.vlines(secret_number, mantisa_exp_corr.min(), mantisa_exp_corr.max(), color='red', label='secret value')
ax.legend()
ax.grid()

In [ ]:
mantisa_exp_corr.sort_values(ascending=False).iloc[:10]

### Correlation sign

In [ ]:
guess_numbers = build_guess_values(component='sign', numbers=mantisa_exp_corr.sort_values(ascending=False).index[:10], guess_range=guess_range)
known_inputs = build_input_values(component='sign')
secret_hw = np.vectorize(hamming_weight)(known_inputs * secret_number)
pd.Series(guess_numbers).sort_values().reset_index(drop=True).plot(figsize=(12,6), marker='^')

In [ ]:
%%time
full_number_corr = compute_corr_numbers(secret_hw, known_inputs, guess_numbers)

In [ ]:
ax = full_number_corr.sort_index().plot(figsize=(15,6), label='correlation')
ax.vlines(full_number_corr.idxmax(), full_number_corr.min(), full_number_corr.max(), color='green', label='max correlation')
ax.vlines(secret_number, full_number_corr.min(), full_number_corr.max(), color='red', label='secret value')
ax.legend()
ax.grid()

In [ ]:
print('Secret value = %f' % secret_number)
full_number_corr.sort_values(ascending=False).iloc[:10]

## Recover a secret number with helper function

In [ ]:
%%time
from weight_recovery_helper import *
full_number_corr = batina_recover_weight(secret_number=secret_number, 
                                         guess_range=guess_range,
                                         mantissa_nbits=mantissa_nbits,
                                         noise=None)
ax = full_number_corr.sort_index().plot(figsize=(15,6), label='correlation')
ax.vlines(full_number_corr.idxmax(), full_number_corr.min(), full_number_corr.max(), color='green', label='max correlation')
ax.vlines(secret_number, full_number_corr.min(), full_number_corr.max(), color='red', label='secret value')
ax.legend()
ax.grid()

In [ ]:
print('Secret value = %f' % secret_number)
full_number_corr.sort_values(ascending=False).iloc[:10]

## Recover multiple secret numbers

In [ ]:
SECRET_VALUE='secret value'
GUESSED_VALUE = 'guessed value'
CORRELATION='correlation'
df_index = [SECRET_VALUE, GUESSED_VALUE, CORRELATION]

In [ ]:
def add_noise_to_signal(signal, snr, freq):
    """
    add noise to the signal
    :param signal: signal
    :param snr: signal to noise ratio 
    :param freq: frequency of noise 
    :return: signal with noise
    """
    sig_max = signal.max()
    sig_min = signal.min()
    return pd.Series(signal).apply(lambda x: x  +
                                   np.average(np.random.normal(0.0, (sig_max-sig_min)/snr, freq)))

In [ ]:
precision = 1e-3
n_secret_numbers = 1000
secret_numbers = np.concatenate((
    np.random.uniform(guess_range[0], guess_range[1], int(n_secret_numbers*2/4) ),
    np.random.uniform(guess_range[0]*3e-1, guess_range[1]*3e-1, int(n_secret_numbers*1/4)),
    np.random.uniform(guess_range[0]*1e-1, guess_range[1]*1e-1, int(n_secret_numbers*1/4))
))
np.random.shuffle(secret_numbers)
signal_to_noise_ratio = 0.5
print('secret_numbers.shape = %s' % (str(secret_numbers.shape)))

## Batina without noise

### load existing DB

In [ ]:
# read the existing data
from os import path
try:
    fname = path.join(TEST_RESULT_DIR, 'batina_results-%d.csv' % mantissa_nbits)
    last_df = pd.read_csv(fname, index_col=0)
    print('read results in %s' % fname)
    print('number of weight values', len(last_df[SECRET_VALUE].unique()))
except FileNotFoundError:
    last_df = pd.DataFrame()
    pass
batina_results = last_df

### main process

In [ ]:
%%time 
for value in secret_numbers:
    if value in batina_results[SECRET_VALUE].unique():
        continue
    s = batina_recover_weight(secret_number=value,
                              guess_range=guess_range,
                              mantissa_nbits=mantissa_nbits)
    s.name = CORRELATION
    df = pd.DataFrame(s).reset_index().rename(columns={'index': GUESSED_VALUE})
    df[SECRET_VALUE] = value
    batina_results = pd.concat([batina_results, df], axis=0, ignore_index=True)

### merge and save DB

In [ ]:
print('number of weight values', len(batina_results[SECRET_VALUE].unique()))
batina_results.to_csv(fname)
print('save results in %s' % fname)

## Batina with noise

### load existing DB

In [ ]:
# read the existing data
from os import path
try:
    fname = path.join(TEST_RESULT_DIR, 'noise_batina_results-%d.csv' % mantissa_nbits)
    last_df = pd.read_csv(fname, index_col=0)
    print('read results in %s' % fname)
    print('number of weight values', len(last_df[SECRET_VALUE].unique()))
except FileNotFoundError:
    last_df = pd.DataFrame()
    pass
noise_batina_results = last_df

### main process

In [ ]:
%%time
# start guessing... - without noise
noise_batina_results = pd.DataFrame(dtype=np.float32, columns=df_index)
for value in secret_numbers:
    if value in noise_batina_results[SECRET_VALUE].unique():
        continue
    s = batina_recover_weight(secret_number=value, 
                              guess_range=guess_range,
                              mantissa_nbits=mantissa_nbits,
                              noise=(add_noise_to_signal, signal_to_noise_ratio, 200))
    s.name = CORRELATION
    df = pd.DataFrame(s).reset_index().rename(columns={'index': GUESSED_VALUE})
    df[SECRET_VALUE] = value
    noise_batina_results = pd.concat([noise_batina_results, df], axis=0, ignore_index=True)

### merge and save DB

In [ ]:
print('number of weight values', len(noise_batina_results[SECRET_VALUE].unique()))
noise_batina_results.to_csv(fname)
print('save results in %s' % fname)

### Statistics

In [ ]:
ERROR = 'error'
def statistics(df):
    dft = df[[SECRET_VALUE, GUESSED_VALUE, CORRELATION]].astype(dtype=np.float64)
    dft[ERROR] = np.abs((dft[GUESSED_VALUE] - dft[SECRET_VALUE])/dft[SECRET_VALUE])    
    best_corr = dft.loc[dft.groupby(SECRET_VALUE)[CORRELATION].idxmax()]
    best_err = dft.loc[dft.groupby(SECRET_VALUE)[ERROR].idxmin()]
    return best_corr.T, best_err.T

In [ ]:
batina_best_corr, batina_best_err = statistics(batina_results)
noise_batina_best_corr, noise_batina_best_err = statistics(noise_batina_results)

In [ ]:
study_score = batina_best_corr
(study_score.T[ERROR] < 1e-3).value_counts()

In [ ]:
error_sorted = study_score.T.sort_values(ERROR, ascending=False)
error_sorted[:10]

In [ ]:
error_sorted.iloc[0][SECRET_VALUE]

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(12, 15))
df = batina_best_corr.T
ax[0].plot(guess_range, guess_range, linewidth=1, label='ideal', color='red')
df[[GUESSED_VALUE, SECRET_VALUE]].set_index(SECRET_VALUE)[GUESSED_VALUE].plot(ax=ax[0], marker='.', label='without noise')
ax[0].set_ylim(np.min(guess_range), np.max(guess_range))

ax[0].set_ylabel('Guessed values')
ax[0].set_title('Guessed values (Batina, best corr)')

df = batina_best_corr.T
df[[ERROR, SECRET_VALUE]].set_index(SECRET_VALUE)[ERROR].plot(ax=ax[1], marker='.', label='without noise')
ax[1].set_ylabel('Error')
ax[1].set_title('Guess Error (Batina, best corr)')

df = batina_best_corr.T
df[[CORRELATION, SECRET_VALUE]].set_index(SECRET_VALUE)[CORRELATION].plot(ax=ax[2], marker='.', label='without noise')
ax[2].set_ylabel('Correlation')
ax[2].set_title('Correlation (Batina, best corr)')

for a in ax:
    a.legend()
    a.grid(True)
    a.set_xlabel('secret values')
    a.set_xlim(np.min(guess_range), np.max(guess_range))

from datetime import datetime
from os import path

fname = path.join(TEST_RESULT_DIR, 
                  'hw-%s.jpg' % (datetime.now().strftime("%Y%m%d_%H%M%S%f")))
plt.savefig(fname)
plt.show()

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(12, 15))
df = batina_best_corr.T
df[[GUESSED_VALUE, SECRET_VALUE]].set_index(SECRET_VALUE)[GUESSED_VALUE].plot(ax=ax[0], marker='.', label='without noise')
df = noise_batina_best_corr.T
ax[0].plot(guess_range, guess_range, linewidth=1, label='ideal', color='red')
df[[GUESSED_VALUE, SECRET_VALUE]].set_index(SECRET_VALUE)[GUESSED_VALUE].plot(ax=ax[0], marker='.', label='with noise')
ax[0].set_ylabel('Guessed values')
ax[0].set_title('Guessed values (Batina, best corr, SNR=%0.2f)' % signal_to_noise_ratio)

df = batina_best_corr.T
df[[ERROR, SECRET_VALUE]].set_index(SECRET_VALUE)[ERROR].plot(ax=ax[1], marker='.', label='without noise')
df = noise_batina_best_corr.T
df[[ERROR, SECRET_VALUE]].set_index(SECRET_VALUE)[ERROR].plot(ax=ax[1], marker='.', label='with noise')
ax[1].set_ylabel('Error')
ax[1].set_title('Guess Error (Batina, best corr, SNR=%0.2f)' % signal_to_noise_ratio)

df = batina_best_corr.T
df[[CORRELATION, SECRET_VALUE]].set_index(SECRET_VALUE)[CORRELATION].plot(ax=ax[2], marker='.', label='without noise')
df = noise_batina_best_corr.T
df[[CORRELATION, SECRET_VALUE]].set_index(SECRET_VALUE)[CORRELATION].plot(ax=ax[2], marker='.', label='with noise')
ax[2].set_ylabel('Correlation')
ax[2].set_title('Correlation (Batina, best corr, SNR=%0.2f)' % signal_to_noise_ratio)

for a in ax:
    a.legend()
    a.grid(True)
    a.set_xlabel('secret values')

from datetime import datetime
from os import path

fname = path.join(TEST_RESULT_DIR, 
                  'nhw-%s.jpg' % (datetime.now().strftime("%Y%m%d_%H%M%S%f")))
plt.savefig(fname)
plt.show()

## New implementation

In [ ]:
import sys
sys.path.append("../")

from pybatina.weight_recovery import *
from pybatina.utils import *

import numpy as np
import pandas as pd

mantissa_three_bytes = [7, 8, 8]
max_mantissa_nbits = np.sum(mantissa_three_bytes)

def build_values(component, mantissa_byte_index):
    if component == 'mantissa':
        assert(mantissa_byte_index < len(mantissa_three_bytes))
        sum_nbits = np.sum(mantissa_three_bytes[:mantissa_byte_index + 1])
        retval = np.left_shift(np.arange(0, 1 << mantissa_three_bytes[mantissa_byte_index]), max_mantissa_nbits - sum_nbits) | np.left_shift(127, max_mantissa_nbits)
    elif component == 'exponent':
        retval = np.left_shift(np.arange(0, 1 << 8), max_mantissa_nbits)
    else:
        raise ValueError('the component is not supported')
    return retval


def build_input_values(component, mantissa_byte_index=None):
    return np.vectorize(int_to_float)(build_values(component, mantissa_byte_index)).astype(np.float32)


def build_guess_values(component, mantissa_byte_index=None, numbers=None):
    values = build_values(component, mantissa_byte_index)
    if numbers is not None:
        if component == 'mantissa':
            int_numbers = np.vectorize(float_to_int)(numbers)
        elif component == 'exponent':
            mask = ~(0xff << max_mantissa_nbits)
            int_numbers = np.vectorize(lambda x: float_to_int(x) & mask)(numbers)
        else:
            raise ValueError('the component is not supported')
        values = np.unique((values | int_numbers[:, np.newaxis]).reshape(-1))
    return np.vectorize(int_to_float)(values).astype(np.float32)


def diff_floats(w, xs):
    iw = float_to_int(w)
    ixs = np.vectorize(float_to_int)(xs)
    diff = np.bitwise_xor(ixs, iw)
    return iw, ixs, diff

### Correlation of mantissa

In [ ]:
%%time
number_of_best_candidates = 20

numbers = None
for mantissa_byte_index in [0, 1, 2]:
    guess_numbers = build_guess_values(component='mantissa', mantissa_byte_index=mantissa_byte_index, numbers=numbers)
    known_inputs = build_input_values(component='mantissa', mantissa_byte_index=mantissa_byte_index)   
    secret_hw = np.vectorize(hamming_weight)(known_inputs * secret_number).astype(np.float32)
    mantisa_corr = compute_corr_numbers(secret_hw, known_inputs, guess_numbers).sort_values(ascending=False)
    numbers = mantisa_corr.index[:number_of_best_candidates]

In [ ]:
ax = mantisa_corr.sort_index().plot(figsize=(15,6), label='correlation', marker='.')
ax.vlines(mantisa_corr.idxmax(), mantisa_corr.min(), mantisa_corr.max(), color='green', label='max correlation')
# ax.vlines(secret_number, mantisa_corr.min(), mantisa_corr.max(), color='red', label='secret value')
ax.legend()
ax.grid()

In [ ]:
mantisa_corr.iloc[:number_of_best_candidates]

### Correlation exponent

In [ ]:
known_inputs = build_input_values(component='exponent')
secret_hw = np.vectorize(hamming_weight)(known_inputs * secret_number)
guess_numbers = build_guess_values(component='exponent')
pd.Series(guess_numbers).sort_values().reset_index(drop=True).plot(figsize=(12,6), marker='.')

In [ ]:
%%time
mantisa_exp_corr = compute_corr_numbers(secret_hw, known_inputs, guess_numbers).sort_values(ascending=False)

In [ ]:
ax = mantisa_exp_corr.sort_index().plot(figsize=(15,6), label='correlation')
ax.vlines(mantisa_exp_corr.idxmax(), mantisa_exp_corr.min(), mantisa_exp_corr.max(), color='green', label='max correlation')
# ax.vlines(secret_number, mantisa_exp_corr.min(), mantisa_exp_corr.max(), color='red', label='secret value')
ax.legend()
ax.grid()

In [ ]:
print('secret_number:', secret_number)
mantisa_exp_corr.iloc[:number_of_best_candidates]

### combine

In [ ]:
int_mantissa = np.vectorize(lambda x: float_to_int(x) & (~(0xff << 23)))(mantisa_corr.index[:number_of_best_candidates])
int_exponent = np.vectorize(float_to_int)(mantisa_exp_corr.index[:number_of_best_candidates])
int_guess_numbers = (int_mantissa | int_exponent[:, np.newaxis]).reshape(-1)
guess_numbers = np.vectorize(int_to_float)(int_guess_numbers).astype(np.float32)

In [ ]:
guess_numbers = np.concatenate((guess_numbers, -guess_numbers))
guess_numbers = guess_numbers[np.where(np.logical_and(guess_numbers >= guess_range[0], guess_numbers <= guess_range[1]))]

In [ ]:
pd.Series(guess_numbers).sort_values().reset_index(drop=True).plot(figsize=(12,6), marker='.')

### Check

In [ ]:
r = 1e2
known_inputs = np.random.uniform(-1e2, 1e2, 1000)
# known_inputs = build_input_values(component='exponent')
secret_hw = np.vectorize(hamming_weight)(known_inputs * secret_number)
check_mantisa_corr = compute_corr_numbers(secret_hw, known_inputs=known_inputs, guess_numbers=guess_numbers).sort_values(ascending=False)

In [ ]:
ax = check_mantisa_corr.sort_index().plot(figsize=(15,6), label='correlation', marker='.')
ax.vlines(check_mantisa_corr.idxmax(), check_mantisa_corr.min(), mantisa_corr.max(), color='green', label='max correlation')
ax.vlines(secret_number, check_mantisa_corr.min(), check_mantisa_corr.max(), color='red', label='secret value')
ax.legend()
ax.grid()

In [ ]:
a = check_mantisa_corr.iloc[:number_of_best_candidates]
a

In [ ]:
iw, ixs, diff = diff_floats(secret_number, a.index)

In [ ]:
print('      {:032b}'.format(iw), '%.010f' % secret_number)
for i in range(len(ixs)):
    s = '{:032b}'.format(diff[i])
    print('[%2d]:' % i, s, '%.010f' % a.index[i], s.count('1'))

In [ ]:
for i in build_input_values('mantissa', 1):
    print(bin(i))
    

In [10]:
import sys
sys.path.append("../")

from pybatina.advanced_weight_recovery import *
from pybatina.utils import *

In [11]:
awr = AdvancedWeightRecovery(guess_range=guess_range)
secret_hamming_weight_set = [np.vectorize(hamming_weight)(input_values * secret_number) for input_values in awr.input_value_set]

In [12]:
%%time
awr.recover_weight(secret_hamming_weight_set)

CPU times: user 1.57 s, sys: 31.9 ms, total: 1.6 s
Wall time: 1.6 s


-4.981213e-01    1.000000
-2.700321e-20    0.996583
-1.159779e-10    0.983851
 4.981213e-01    0.962515
-7.600728e-06    0.957191
 2.700321e-20    0.954030
 1.159779e-10    0.935469
 7.600728e-06    0.902455
-4.981290e-01    0.809400
-4.981175e-01    0.800439
dtype: float64